In [1]:
import wandb
from dotenv import load_dotenv
import os
import numpy as np
import json

import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard # type: ignore
from tensorflow.keras import layers, models # type: ignore
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint
from sklearn.model_selection import train_test_split
import sys
sys.path.append(os.path.abspath('../'))
import utils.tuner_models as tuner_models
import utils.dataset_loader as dataset_loader
# Load the .env file
load_dotenv()

2025-05-08 12:13:20.407764: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-08 12:13:20.417713: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-08 12:13:20.446016: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746699200.483683    5738 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746699200.495684    5738 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746699200.527933    5738 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

True

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


2025-05-08 12:13:31.639945: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [3]:
# Optionally fetch the key (for debugging or explicit control)
wandb_api_key = os.getenv("WANDB_API_KEY")

# Check if the key is available
if wandb_api_key is None:
    print("WANDB_API_KEY not found in environment variables.")
else:
    wandb.login(key=wandb_api_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/miroslav/.netrc
wandb: Currently logged in as: xtodorovic (mt-thesis) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
# Paths 
DATASETS_PATH = '../datasets'
random_dataset = dataset_loader.load_dataset_files_with_cache(DATASETS_PATH, cache_path=f"{DATASETS_PATH}/cache/random_dataset_cache.pkl")

Checking for cache at: ../datasets/cache/random_dataset_cache.pkl
Loading datasets from cache: ../datasets/cache/random_dataset_cache.pkl


In [5]:
traces = dataset_loader.get_trace_matrix(random_dataset)  # shape (n_traces, n_samples)
print(f"Loaded traces: {traces.shape}")  # (n_traces, n_pois, 1)


Loaded traces: (10000, 5000)


In [6]:
selected_pois = np.load(os.path.join('../dataset/present/pois', "nibble_1_pois.npy"))
labels_path = os.path.join('../dataset/present', f"nibble_0_labels.npy")

labels = np.load(labels_path)

In [7]:
def train_nibble_models(dataset_dir, build_fn, description="advanced_with_nibble_pois", project_name="present-sca", model_out_dir="models/", nibble_idx=None):
    """
    Train model(s) for PRESENT nibble leakage.

    Parameters:
        dataset_dir (str): Path to reduced_traces.npy and nibble_X_labels.npy
        build_fn (function): Keras model builder (for Keras Tuner)
        project_name (str): wandb project name
        model_out_dir (str): Where to save models
        nibble_idx (int or None): If set, trains only that nibble. If None, trains all 16.
    """
    
    

    os.makedirs(model_out_dir, exist_ok=True)

    nibble_range = [nibble_idx] if nibble_idx is not None else range(16)

    for idx in nibble_range:
        print(f"\nTraining model for nibble {idx}...")

        selected_pois = np.load(os.path.join('../dataset/present/pois', f"nibble_{idx}_pois.npy"))
        labels_path = os.path.join('../dataset/present', f"nibble_{idx}_labels.npy")

        labels = np.load(labels_path)
        traces = dataset_loader.get_trace_matrix(random_dataset) 
        traces = traces[:, selected_pois]

        if len(traces.shape) == 2:
            traces = np.expand_dims(traces, axis=-1)
        print(f"Loaded traces: {traces.shape}")  # (n_traces, n_pois, 1)
        x_train, x_val, y_train, y_val = train_test_split(traces, labels, test_size=0.2, random_state=42)

        trace_length = len(traces[0])
        run = wandb.init(
            entity="mt-thesis",
            project=project_name,
            name=f"nibble_{idx}_cnn_{description}",
            config={"nibble": idx},
            reinit=True
        )

        tuner = kt.Hyperband(
            build_fn,
            objective='val_accuracy',
            max_epochs=20,
            directory='kerastuner_logs_4',
            project_name=f'nibble_{idx}_tuning_cnn_{description}'
        )

        tuner.search(
            x_train, y_train,
            validation_data=(x_val, y_val),
            epochs=20,
            callbacks=[
                WandbMetricsLogger(),
                WandbModelCheckpoint(filepath=os.path.join(run.dir, "best_model.keras"))
            ]
        )

        best_model = tuner.get_best_models(num_models=1)[0]
        best_model.save(os.path.join(model_out_dir, f"nibble_{idx}_{description}_model.keras"))

        best_hp = tuner.get_best_hyperparameters(1)[0].values
        with open(os.path.join(model_out_dir, f"nibble_{idx}_{description}_hp.json"), "w") as f:
            json.dump(best_hp, f, indent=4)

        run.finish()

In [8]:
DATASET_DIR = "../dataset/present"

In [9]:
num_classes = 16

In [ ]:
train_nibble_models(DATASET_DIR, tuner_models.build_advanced_cnn, nibble_idx=0)

Trial 26 Complete [00h 03m 20s]
val_accuracy: 0.3305000066757202

Best val_accuracy So Far: 0.48649999499320984
Total elapsed time: 00h 40m 02s


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch/accuracy,▁▂▁▁▂▁▁██▁▁▁▁▁▁▂▃▆████▁▁▁▇████▁▁▁▁▁▁▅▆▇█
epoch/epoch,▂▁▁▁▁▁▂▃▂▂▂▃▃▂▂▁▂▃▁▂▂▆▄▆▁▃▄▅█▃▇▇▇█▁▄▄▃▄▇
epoch/learning_rate,▁▁▂▂▁█▂▁██▂████▁▁▁▁▁▂███████▂▂▂▂███▁▁▁▁▁
epoch/loss,▇█▇▇▆▇▇▄▂▇▅▇▁▇▇▇▇▆▅▇▇▇▇▅▂▁▁▇▇▇▁▁█▇▇▇▇▅▄▁
epoch/val_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▄▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁
epoch/val_loss,▁▁▁▁▁▂▆▄▁▂▁▁▁▁▁▁▁▁▁▇▁▁▁▁▁▁▁▃▅█▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,0.95675
epoch/epoch,19
epoch/learning_rate,0.0001
epoch/loss,0.22225
epoch/val_accuracy,0.072


: 